# Co-Verification tricks

In some cases, you might want to prove that a python function, executed natively, does the same as its synthesized counterpart. This is in particular the case for functions that are reused often.

Take a `@rtl_function` example:

In [1]:
from cyhdl import *

In [2]:
class often_used:
    @rtl_function
    def funclet(rtl, en, a, b):
        if en == True:
            b.next = ~a
        else:
            b.next = a

We will be calling this function twice:
1. From the hardware context, like a function
2. From a native context, using `yield from`

We create a specific co-simulation design. It will only function with a cosimulation backend, such as CXXRTL.

In [3]:
from yosys.simulator import CXXRTL
from myirl.library.basictypes import Bool

class CoSimDesign(cyrite_factory.Module):
    def __init__(self):
        super().__init__("cosim", CXXRTL)
        
    @cyrite_factory.block_component
    def unit(self,
             clk : ClkSignal,
             en: Bool,
             a : Signal,
             b : Signal.Output):
        
        @always(clk.posedge)
        def worker():
            # Call @rtl_function like a function from a rtl context:
            often_used.funclet(en, a, b)

        return worker

    @cyrite_factory.testbench("ns")
    def testbench(self):
        clk = self.ClkSignal(name = 'clk')
        en = self.Signal(bool(), name = 'en')
        a, b = [ self.Signal(intbv()[8:], name = n) for n in "ab" ]
        co_a, co_b = [ self.Signal(intbv()[8:], name = n) for n in "AB" ]
        verify = self.Signal(bool())
        
        continuous_assignments = [
            co_a   @assign@  a
        ]

        uut = self.unit(clk, en, a, b)

        # # Co-Unit:
        @self.always(clk.posedge)
        def co_unit():
            # Call the same funclet using `yield from`:
            yield from often_used.funclet(en, co_a, co_b)

        @self.always(clk.posedge)
        def verification():
            if verify:
                print("Verify:", en, self.now(), co_b, b)
                assert co_b == b
        
        @self.always(delay(1))
        def clkgen():
            clk.next = ~clk

        @self.sequence
        def main():
            verify.next = False
            yield delay(20)
            verify.next = True
            for v in [0, 0xaa, 0x55, 0x2d, 0x85]:
                en.next = (v & 1) != 0
                yield clk.negedge
                a.next = v

            raise StopSimulation

        return instances()

We instance the design and run the test bench.
If we do not make changes on the hardware function, you can set `recompile` to False. This can be useful if you develop test benches for large hardware designs.

In [4]:
d = CoSimDesign()
tb = d.testbench()
tb.run(2000, wavetrace = 'test.vcd', recompile = True)

 Declare obj 'unit' in context '(CoSimDesign 'cosim')'(<class '__main__.CoSimDesign'>) 
 Adding module with name `unit` 
 FINALIZE implementation `unit` of `unit` 
Compiling /tmp/myirl_cosim_dzt9gq1z/unit_8133.pyx because it changed.
[1/1] Cythonizing /tmp/myirl_cosim_dzt9gq1z/unit_8133.pyx
running build_ext
building 'runtime.unit_8133' extension
creating build/temp.linux-x86_64-3.10/tmp/myirl_cosim_dzt9gq1z
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=unit_8133 -Iruntime -I/tmp/myirl_cosim_dzt9gq1z/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/myirl_cosim_dzt9gq1z/unit_8133.cpp -o build/temp.linux-x86_64-3.10/tmp/myirl_cosim_dzt9gq1z/unit_8133.o
gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -DCOSIM_NAMESPACE=unit_8133 -Iruntime -I/tmp/myirl_cosim_dzt9gq1z/ -I/usr/share/yosys/include/backends/cxxrtl/runtime -I/usr/local/include/python3.10 -c /tmp/my

Using '/tmp/myirl_cosim_dzt9gq1z/' for output
Cosimulation: co_a not connected to backend
Cosimulation: co_b not connected to backend
Cosimulation: verify not connected to backend
STOP SIMULATION @30


## Detailed explanations

What happens here is the following:
* `self.unit` is compiled as CXXRTL back end module and is imported ad-hoc
* The signals passed to its interface are connected to the backend, i.e. they are consumed and driven by the latter.
* The `co_*` signals are not connected to the back end and are thus handled by the cosimulation layer upon occuring events, such as a `clk.posedge`.
* All functionality in the `testbench()` function runs as native python. Therefore one has to ensure that calls to macros are evaluated explicitely or the `yield from` constructs are used for context sensitive `@rtl_function`s or `@cyrite_method`s.

To determine from the code, in which domain the code is executed or evaluated, the thumb rule applies:
* `self.always(...)` runs here in native execution
* `always(...)` is transpiled to hardware

Note that Co-Processes running in the native Python context pretty much allow any kind of Python extension looped in and only a small built-in sub set of Python can be transpiled to HDL.

## Performance aspects: Event signals

A Co-Simulator is normally the driving force in a design, creating the external stimuli for a pure digital design without functional delay simulation.

All the above `@self.always` co-processes will only react to external stimuli caused by `EventSignal` types. The reason for this is performance: all combinatorial signal dependencies of their sources are sorted out by the faster back end. The co-simulation front end should only have to set a few signals, provide a clock and a reset.

This implies certain restrictions:
* No clock generators inside the synthesized RTL, for instance, PLLs can not be simulated this way.
* co-processes can only use EventSignal types in their sensitivity list
* `@sequence` functions can only wait for events of EventSignals as well, i.e. constructs such as `s.posedge` will only work for a clock signal type.

Non-Event signals, like outputs of the simulation of the compiled backend unit will thus have to be polled explicitely for changes.

### Native simulation issues

Note that the Co-Simulation layer is very rudimentary and does not allow to simply run a hardware design on the native co-simulation side.

In particular, the support for co-processes is limited to:
* Direct assignments of signals, no combinatorial logic
* `@self.always()` only allowed for:
    * Event types caused by EventSignal types (ClkSignal, ResetSignal, ..)
    * Delay arguments (`delay(cycles)`) where cycles must be integer
    * Co-processes inside the testbench top, i.e. no hierarchy

Also keep in mind that the current Co-Simulator only allows **one** unit under test instance.